In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install OpenDartReader

In [ ]:
import OpenDartReader
import pandas as pd

api_key = 'e81e10a7e5267be70f67c42da4dd3b06a69abd48' # API Key 입력

stock_name = 'LG전자'
dart = OpenDartReader(api_key)
df1 = pd.DataFrame()
# LG전자 2016년의 1분기보고서 전체항목 불러오기
df1 = df1.append(dart.finstate_all(stock_name, 2016, reprt_code='11013', fs_div='CFS'))
print(df1) # 결과를 터미널에 보여준다.

# to_excel() 시, 파일 경로까지 함께 넣어주면 더 관리하기 좋다.
#fileName = 'C:/Projects/result_LG전자.xlsx'

#df1.to_excel(excel_writer=fileName)


reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
           rcept_no reprt_code bsns_year corp_code sj_div  sj_nm  \
0    20160516002710      11013      2016  00401731     BS  재무상태표   
1    20160516002710      11013      2016  00401731     BS  재무상태표   
2    20160516002710      11013      2016  00401731     BS  재무상태표   
3    20160516002710      11013      2016  00401731     BS  재무상태표   
4    20160516002710      11013      2016  00401731     BS  재무상태표   
..              ...        ...       ...       ...    ...    ...   
202  20160516002710      11013      2016  00401731    SCE  자본변동표   
203  20160516002710      11013      2016  00401731    SCE  자본변동표   
204  20160516002710      11013      2016  00401731    SCE  자본변동표   
205  20160516002710      11013      2016  00401731    SCE  자본변동표   
206  20160516002710      11013      2016  00401731    SCE  자본변동표   

                               account_id account_nm  \
0                      ifrs_CurrentAssets       유동자산   
1             ifrs_C

<ipython-input-13-9de42dbaaee3>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(dart.finstate_all(stock_name, 2016, reprt_code='11013', fs_div='CFS'))


In [ ]:
import OpenDartReader
import pandas as pd
import time

api_key = '59f17375e15ffafc9798fdae76fe437a382736c9' # OpenDart API 에서 받는 KEY 입력

# 얻고자 하는 종목명 리스트 형태로 입력
stock_names = ['삼성전자', 'LG전자', '현대자동차']
dart = OpenDartReader(api_key)

# 데이터 초기화. df2 는 정리된 데이터를 모으는 곳이며 얻고자 하는 데이터 이름을
# column 화 시킴.  빈 index 를 넣으면 추가가 안되므로 임으로 하나 넣자.
df2 = pd.DataFrame(columns=['유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                      '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

# '11013'=1분기보고서, '11012' =반기보고서, '11014'=3분기보고서, '11011'=사업보고서
reprt_code = ['11013', '11012', '11014', '11011']

# 데이터를 얻기위한 반복문 시작
for stocks in stock_names:
    # 정리 완료된 파일 저장하기 위한 경로 및 이름. result_종목이름.xlsx 형태로 저장된다.
    fileName = f'C:/Projects/stocks_sorting/data/result_{str(stocks)}.xlsx'
    for i in range(2015, 2022): # OpenDart는 2015년부터 정보를 제공한다.

        # 더미 리스트 초기화. 1 ~ 4 분기 데이터를 합할 예정이므로 4 크기 만큼의 리스트 선언.
        current_assets = [0, 0, 0, 0] # 유동자산
        liabilities = [0, 0, 0, 0] # 부채총계
        equity = [0, 0, 0, 0] # 자본총계
        revenue = [0, 0, 0, 0] # 매출액
        grossProfit = [0, 0, 0, 0] # 매출총이익
        income = [0, 0, 0, 0] # 영업이익
        net_income = [0, 0, 0, 0] # 당기순이익
        cfo = [0, 0, 0, 0] # 영업활동현금흐름
        cfi = [0, 0, 0, 0] # 투자활동현금흐름
        fcf = [0, 0, 0, 0] # 잉여현금흐름 : 편의상 영업활동 - 투자활동 현금흐름으로 계산

        for j, k in enumerate(reprt_code):
            df1 = pd.DataFrame() # Raw Data
            if str(type(dart.finstate_all('004840', i, reprt_code=k, fs_div='CFS'))) == "<class 'NoneType'>":
                pass

            # 타입이 NoneType 이 아니면 읽어온다.
            else:
                df1 = df1.append(dart.finstate_all(stock_name, i, reprt_code=k, fs_div='CFS'))
                # 재무상태표 부분
                condition = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산') # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계') # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & \
                            ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & ((df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & \
                                ((df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & \
                                ((df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') | \
                                (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') | \
                                (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익')| (df1.account_nm == '연결당기순이익')|(df1.account_nm == '연결분기(당기)순이익')|(df1.account_nm == '연결반기(당기)순이익')|\
                                (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & ((df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))
                condition_9 = (df1.sj_nm == '현금흐름표') & ((df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                current_assets[j] = int(df1.loc[condition].iloc[0]['thstrm_amount'])
                liabilities[j] = int(df1.loc[condition_2].iloc[0]['thstrm_amount'])
                equity[j] = int(df1.loc[condition_3].iloc[0]['thstrm_amount'])
                revenue[j] = int(df1.loc[condition_4].iloc[0]['thstrm_amount'])
                grossProfit[j] = int(df1.loc[condition_5].iloc[0]['thstrm_amount'])
                income[j] = int(df1.loc[condition_6].iloc[0]['thstrm_amount'])
                net_income[j] = int(df1.loc[condition_7].iloc[0]['thstrm_amount'])
                cfo[j] = int(df1.loc[condition_8].iloc[0]['thstrm_amount'])
                cfi[j] = int(df1.loc[condition_9].iloc[0]['thstrm_amount'])
                fcf[j] = (cfo[j] - cfi[j])

                if k == '11013': # 1분기.
                    path_string = str(i) + '-03-31'
                elif k == '11012': # 2분기
                    path_string = str(i) + '-06-30'
                elif k == '11014': # 3분기
                    path_string = str(i) + '-09-30'
                else: # 4분기. 1 ~ 3분기 데이터를 더한다음 사업보고서에서 빼야 함
                    path_string = str(i) + '-12-30'
                    revenue[j] = revenue[j] - (revenue[0] + revenue[1] + revenue[2])
                    grossProfit[j] = grossProfit[j] - (grossProfit[0] + grossProfit[1] + grossProfit[2])
                    income[j] = income[j] - (income[0] + income[1] + income[2])
                    net_income[j] = net_income[j] - (net_income[0] + net_income[1] + net_income[2])
                    fcf[j] = fcf[j] - (fcf[0] + fcf[1] + fcf[2])

                # 데이터프레임에 저장하는 부분
                df2.loc[path_string] = [current_assets[j], liabilities[j], equity[j],
                                    revenue[j], grossProfit[j], income[j], net_income[j], cfo[j], fcf[j]]
                df2.tail() # 데이터프레임 끝에 저장한다.
            # 너무 잦은 요청이 있을 경우 OpenDart API 측에서 IP 를 차단하니 텀을 둔다.
            time.sleep(0.1)

    # 원본 dataframe 에도 영향을 끼치게끔 (inplace=True) 첫 행 drop.
    df2.drop(['1900-01-01'], inplace=True) # 첫 행 drop

    # 파일 저장. fileName 에 명시된 경로에 각 종목코드별로 다른 이름으로 저장
    df2.to_excel(fileName)

    # 그 다음 종목의 데이터를 얻기 위해 df2 재초기화
    df2 = pd.DataFrame(columns=['유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01']) # 정리된 Data. 빈 index 를 넣으면 추가가 안되므로 하나 넣자.


reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}



<ipython-input-14-17acc61683df>:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(dart.finstate_all(stock_name, i, reprt_code=k, fs_div='CFS'))


AttributeError: ignored

In [ ]:
   # 재무상태표 부분
                condition = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산') # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계') # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & \
                            ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & ((df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & \
                                ((df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & \
                                ((df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') | \
                                (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') | \
                                (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익')| (df1.account_nm == '연결당기순이익')|(df1.account_nm == '연결분기(당기)순이익')|(df1.account_nm == '연결반기(당기)순이익')|\
                                (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & ((df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))
                condition_9 = (df1.sj_nm == '현금흐름표') & ((df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

In [ ]:
#어닝프라이즈.. 특징주
import requests
from bs4 import BeautifulSoup
import datetime
import re

dates = []
codes = []
titles = []

pat_1 = re.compile('[\]\)>]+[ ]*[\D]+[,]{1}')  #[특징주]OO,...
pat_2 = re.compile('[\]\)>]+[ ]*[가-힣a-zA-Z&]+[ ]{1}') #[특징주]OO ...
pat_3 = re.compile("' [가-힣a-zA-Z&]+ ") # <특징주> '어닝 서프라이즈' OO
pat_sub = re.compile("[^가-힣a-zA-Z& ]")

for page_num in range(1,42,1):
    url = 'https://finance.naver.com/news/news_search.nhn?'\
          'rcdate=1&q=%C6%AF%C2%A1%C1%D6+%BE%EE%B4%D7+%BC%AD%C7%C1%B6%F3%C0%CC%C1%EE&'\
          'x=9&y=15&sm=title.basic&pd=4&stDateStart=1997-01-01&stDateEnd=2019-12-31&'\
          'page='+ str(page_num)

    raw = requests.get(url).text
    html = BeautifulSoup(raw, 'html.parser')

    html_titles = html.select('.articleSubject')
    html_dates = html.select('.wdate')

    for html_title in html_titles:
        title = html_title.text.strip()
        code = pat_1.search(title)
        if code is not None:
            code = pat_sub.sub(' ', code.group()).strip()
            codes.append(code)
            continue

        code = pat_2.search(title)
        if code is not None:
            code = pat_sub.sub(' ', code.group()).strip()
            codes.append(code)
            continue

        code = pat_3.search(title)
        if code is not None:
            code = pat_sub.sub(' ', code.group()).strip()
            codes.append(code)
            continue

        codes.append(code)


    for html_date in html_dates:
        date = datetime.datetime.strptime(html_date.text.strip()[:10], '%Y-%m-%d')
        dates.append(date)

print('crowling done.')

with open('earning.txt', 'w') as file:
    for idx in range(len(codes)):
        file.write('{0} / {1}\n'.format(codes[idx], dates[idx]))

# 새 섹션